In [7]:
import re
import os
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json
import subprocess
from shutil import copytree, ignore_patterns

In [8]:
from core.compilers import CompilationStatus
from core.compilers import drizzle as drizzle_compiler
from core.compilers import typespec as typespec_compiler

In [9]:
class Builder:
    def __init__(self, client: AnthropicBedrock, template_dir: str = "templates"):
        self.client = client
        self.template_dir = template_dir
        self.jinja_env = jinja2.Environment()
        self.typespec_tpl = self.jinja_env.from_string(stages.typespec.PROMPT)
        self.drizzle_tpl = self.jinja_env.from_string(stages.drizzle.PROMPT)
        self.router_tpl = self.jinja_env.from_string(stages.router.PROMPT)
        self._model = "anthropic.claude-3-5-sonnet-20241022-v2:0"

    def build(self, application_description: str, out_dir: str = "app_output"):
        copytree(self.template_dir, out_dir, ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'
        compiler_typespec = typespec_compiler.TypeSpecCompiler(out_dir)
        prompt_typespec = self.typespec_tpl.render(
            application_description=application_description,
        )
        typespec_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            messages=[{"role": "user", "content": prompt_typespec}]
        )
        typespec_definitions = stages.typespec.parse_output(typespec_response.content[0].text)
        typespec_result = compiler_typespec.compile(typespec_definitions["typespec_definitions"])
        print("finished typespec")
        #if typespec_result["result"] != CompilationStatus.SUCCESS:
        #    raise Exception("Failed to compile typespec")
        
        compiler_drizzle = drizzle_compiler.DrizzleCompiler(out_dir)
        prompt_drizzle = self.drizzle_tpl.render(
            typespec_definitions=typespec_definitions["typespec_definitions"],
        )
        drizzle_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            messages=[{"role": "user", "content": prompt_drizzle}]
        )
        drizzle_schema = stages.drizzle.parse_output(drizzle_response.content[0].text)
        drizzle_result = compiler_drizzle.compile(drizzle_schema["drizzle_schema"])
        print("finished drizzle")
        #if drizzle_result["result"] != CompilationStatus.SUCCESS:
        #    raise Exception("Failed to compile drizzle")
        
        prompt_router = self.router_tpl.render(
            typespec_definitions=typespec_definitions["typespec_definitions"],
            user_request=application_description,
        )
        router_response = self.client.messages.create(
            model=self._model,
            max_tokens=8192,
            tools=stages.router.TOOLS,
            messages=[{"role": "user", "content": prompt_router}]
        )
        print("ROUTER_RESPONSE", router_response)
        router_result = stages.router.parse_outputs([
            content for content in router_response.content
        ])["user_functions"]
        return {
            "typespec": typespec_result,
            "drizzle": drizzle_result,
            "router": router_result
        }

In [10]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [11]:
builder = Builder(client, template_dir="templates")

In [12]:
app_result = builder.build(
    "Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines for specific list of available equipment and time constraints.",
    out_dir="app_output"
)

finished typespec
start checking drizzle
end checking drizzle
finished drizzle
ROUTER_RESPONSE Message(id='msg_bdrk_01PpB43TDVkE6HhvExv66rJA', content=[TextBlock(text="I'll help analyze the TypeSpec definition and extract user functions for the gym tracking bot. I'll use the extract_user_functions tool to structure the functions according to the schema.", type='text'), ToolUseBlock(id='toolu_bdrk_01F2ndXmjk1PrfhTG99v2Jr8', input={'user_functions': [{'name': 'recordExercise', 'description': 'Log a completed exercise with details including number of sets, reps, weight used, and equipment. Used when the user wants to record their workout activity.', 'examples': ['I did 3 sets of bench press with 135lbs, 12 reps each', 'Just finished 4 sets of squats at 185lbs', 'Completed 3 sets of pull-ups, 8 reps each', 'Today I did deadlifts: 225lbs for 5 reps, 3 sets']}, {'name': 'suggestRoutine', 'description': "Generate a workout routine based on user's available time, equipment, and optionally targ

In [13]:
app_result

{'typespec': {'result': <CompilationStatus.SUCCESS: 1>,
  'errors': None,
  'stdout': 'TypeSpec compiler v0.64.0\n\nCompilation completed successfully.\n\n'},
 'drizzle': {'result': <CompilationStatus.FAILURE: 2>,
  'errors': "Cannot find module 'dotenv/config'\nRequire stack:\n- /home/eugenek/neon/bot-new/agent/app_output/app_schema/drizzle.config.ts\n- /home/eugenek/.npm/_npx/7c7555b0b81cc7e0/node_modules/drizzle-kit/bin.cjs\n",
  'stdout': "No config path provided, using default 'drizzle.config.ts'\nReading config file '/home/eugenek/neon/bot-new/agent/app_output/app_schema/drizzle.config.ts'\n"},
 'router': [{'name': 'recordExercise',
   'description': 'Log a completed exercise with details including number of sets, reps, weight used, and equipment. Used when the user wants to record their workout activity.',
   'examples': ['I did 3 sets of bench press with 135lbs, 12 reps each',
    'Just finished 4 sets of squats at 185lbs',
    'Completed 3 sets of pull-ups, 8 reps each',
    '